In [37]:
import csv
import json
import requests
from bs4 import BeautifulSoup
import numpy as np
import time
import os



In [38]:
class websiteData:
    def __init__(self, str_reviews=[], str_reviews_title=[], str_reviews_date=[], full_star = [], empty_star = [],
                 reviews_hrefs=[], reviews_text_links=[], reviews_full_links=[], site_headers={}, rev_api_link = '', reviews_rating_data = [],
                 website_main_link='', reviews_title_links=[], reviews_content_links=[], website_name = '', api_total_elements = '',
                 reviews_and__links_on_them={}, website_link='', reviews_posting_date_links=[], number_of_stars = [], review_status = []):

        self.revTxt = str_reviews
        self.revTitle = str_reviews_title
        self.revDate = str_reviews_date
        self.htmlClasses = {
            "revTxt": reviews_text_links,
            "revFull": reviews_full_links,
            "revTitle": reviews_title_links,
            "revContent": reviews_content_links,
            "revDate": reviews_posting_date_links
        }
        self.links = {
            "allWebsite": website_main_link,
            "websiteUrl": website_link,
            "revLinks": reviews_hrefs,
            "api": rev_api_link
        }
        self.api = {
            "totalElem": api_total_elements
        }
        self.revPlusOtherData = reviews_and__links_on_them
        self.headers = site_headers
        self.name = website_name
        self.rating = {
            "star": full_star,
            "emptyStar": empty_star,
            "ratingNum": number_of_stars,
            "revStatus": review_status
        }
        self.revRating = reviews_rating_data

In [39]:
def data_getting(url, text_class, need_href, w):    #w -- website (class)
    r = requests.get(url, headers=w.headers)
    soup = BeautifulSoup(r.text, 'lxml')
    reviews_text = soup.find_all(class_=w.htmlClasses[text_class][0])
    for q in range(1, len(w.htmlClasses[text_class])):
        for i, rev in enumerate(reviews_text):
            reviews_text[i] = rev.find(class_=w.htmlClasses[text_class][q])
            #print(reviews_text[i])

    c = 0
    reviews_clean = [[], []]
    reviews_rating = [[], []]

    if need_href:
        reviews_hrefs = [[], []]
        for review in reviews_text:
            if review != None:
                if w.revRating[c] > 3:
                    co = 1
                else:
                    co = 0
                reviews_clean[co].append(review.text)
                reviews_rating[co].append(w.revRating[c])
                try:
                    reviews_hrefs[co].append(w.links[allWebsite] + review.get("href"))
                except Exception:
                    reviews_hrefs[co].append("None")
                c += 1
        return reviews_clean, reviews_rating, reviews_hrefs

    else:
        for review in reviews_text:
            if review != None:
                if w.revRating[c] > 3:
                    co = 1
                else:
                    co = 0
                reviews_clean[co].append(review.text)
                reviews_rating[co].append(w.revRating[c])
                c += 1
        return reviews_clean, reviews_rating

In [40]:
def convert(reviews):
    review_converted = []
    for review in reviews:
        for i in review:
            map(str, i)
        review = ''.join(review)
        review_converted.append(review)
    return review_converted

In [41]:
def rating_getting(url, w):
    r = requests.get(url, headers=w.headers)
    soup = BeautifulSoup(r.text, 'lxml')
    reviews_text = soup.find_all(class_=w.rating[star])
    for q in range(1, len(w.rating[star]) - 1):
        for i, rev in enumerate(reviews_text):
            reviews_text[i] = rev.find(class_=w.rating[star][q])

    reviews_rating = []
    for rev in reviews_text:
        q = rev.find_all(class_=w.rating[star][-1])
        #print(q)
        if len(q) != 0:
            reviews_rating.append(len(q))
    return reviews_rating

In [42]:
# def parsing(url, status, path, text_class, w):    # w -- website (class)
#     print('--- Parsing in process... ---')
#     delays = [1, 2, 1.5]
#     #delays = [11, 12, 13, 11.5, 12.5, 13.5, 11.2, 12.3, 11.8]
#     time.sleep(np.random.choice(delays))
#     addData = []
#     for _ in range(1):
#         for classI in text_class:
#             received_data = data_getting(url, classI, False, w)
#             if received_data == []:
#                 break
#         # Eсли папок с классами не существует, программа их создаст
#         # if not os.path.exists(path + fr'{w.name}\{status}'):
#         #     os.makedirs(path + fr'{w.name}\{status}')
#             converted_data = convert(received_data)
#             dataSize = len(converted_data)
#             #for i, review in enumerate(converted_data):
#             for dta in converted_data:
#
#
#                 time.sleep(np.random.choice(delays))
#     with open(path + f'{w.name}_{status}.json', 'w', encoding='utf-8') as file:
#         json.dump(addData, file, indent=4, ensure_ascii=False)

In [43]:
# data receiving
def review_other_data(titles, data, w):
    pos, neg = "Negative", "Positive"
    links_to_the_reviews = {
        pos: {},
        neg: {}
    }

    for e in range(2):
        if e == 1:
            st = neg
        else:
            st = pos

        for i in range(len(titles[e])):
            links_to_the_reviews[st][titles[e][i]] = {}
            for j in range(len(data)):
                if data[j] == []:
                    links_to_the_reviews[st][titles[e][i]][dataTypes[j]] = 'None'
                else:
                    links_to_the_reviews[st][titles[e][i]][dataTypes[j]] = data[j][e][i]

    if not os.path.exists(r'result\from_html'):
        os.makedirs(r'result\from_html')
    with open(fr'result\from_html\revData_{w.name}.json', 'w', encoding='utf-8') as file:
        json.dump(links_to_the_reviews, file, indent=4, ensure_ascii=False)
    #return links_to_the_reviews

In [44]:
# var/val initialization

revTxt, revFull, revTitle, revContent, revDate = "revTxt", "revFull", "revTitle", "revContent", "revDate"
allWebsite, revLinks, websiteUrl, api, totalElem = "allWebsite", "revLinks", "websiteUrl", "api", "totalElem"
star, emptyStar, ratingNum, revStatus = 'star', 'emptyStar', 'ratingNum', 'revStatus'

sravni_ru, rustore = websiteData(), websiteData()

dataTypes = ['Title', 'Link', 'Date', 'Rating']

rev_websites = {
    sravni_ru: r"https://www.sravni.ru/bank/tinkoff-bank/otzyvy/",
    rustore: r"https://apps.rustore.ru/app/com.idamob.tinkoff.android/reviews"
}

urles_api = {
    sravni_ru: r'https://www.sravni.ru/proxy-reviews/reviews/?filterBy=withRates&fingerPrint=eab9ea9d6cdaf03fcbbff091af5c0195&isClient=false&locationRoute=&newIds=true&orderBy=byDate&pageIndex=1&pageSize=10&reviewObjectId=5bb4f769245bc22a520a6353&reviewObjectType=banks&specificProductId=&withVotes=true',
    rustore: r'https://backapi.rustore.ru/comment/comment?packageName=com.idamob.tinkoff.android&sortBy=NEW_FIRST&pageSize=100&pageNumber=0'
}

In [45]:
# sravni.ru init

sravni_ru.links[websiteUrl] = rev_websites[sravni_ru]
sravni_ru.links[allWebsite] = r'https://www.sravni.ru/'

sravni_ru.htmlClasses[revTxt] = ['review-card_link__b9zg3']
sravni_ru.htmlClasses[revFull] = ['review-card_wrapper__gnPSK common_blockWrapper__Fw5ZV']
sravni_ru.htmlClasses[revTitle] = ['review-card_title__zYdxx articleTypography_article-h3__wuxLw']
sravni_ru.htmlClasses[revContent] = ['review-card_text__jTUSq articleTypography_article-comment__Px4n0 h-mt-8 h-mb-8 review-card_text__in-list__DsDRg']
sravni_ru.htmlClasses[revDate] = ['h-mb-20 _1n8o0h2 _vea58f _1ivat6c _pbfp49', '_1n8o0h2 _vea58f _1ivat6c', '_1n8o0h2 _vea58f']

sravni_ru.rating[star] = ['h-mb-20 _1n8o0h2 _vea58f _1ivat6c _pbfp49', '_n6sdlh _akgnn3 _8136q1 _1xowbbl _18zu2v3 _g18kgu shape-round']


# for num in range(1, 2482):
#     q, sravni_ru.links[revLinks] = load_data(sravni_ru.links[websiteUrl] + f"&pageIndex={num}", sravni_ru.htmlClasses[revTxt], sravni_ru.links[allWebsite], True)
#
#     for i in q:
#         sravni_ru.revBad.append(i)

sravni_ru.name = "sravni.ru"

sravni_ru.headers = {
    "Accept": "*/*",
    "User-Agent": r"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 YaBrowser/24.1.0.0 Safari/537.36"
}

In [46]:
# rustore.ru init

rustore.links[websiteUrl] = rev_websites[rustore]
rustore.links[allWebsite] = 'https://apps.rustore.ru/'
rustore.links[api] = 'https://backapi.rustore.ru/comment/comment?packageName=com.idamob.tinkoff.android&sortBy=NEW_FIRST&pageSize=1000&pageNumber='
rustore.api[totalElem] = 'totalElements'

rustore.htmlClasses[revTxt] = ['ReviewItems_comment__W6IzJ']
rustore.htmlClasses[revFull] = ['ReviewItems_wrapperItem__edebG']
rustore.htmlClasses[revTitle] = ['None']
rustore.htmlClasses[revDate] = ['ReviewItems_date__XJhw_']
rustore.htmlClasses[revContent] = ['ReviewItems_comment__W6IzJ']

rustore.rating[star] = ['']

rustore.name = "rustore.ru"

rustore.headers = {
    "Accept": "*/*",
    "User-Agent": r"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 YaBrowser/24.1.0.0 Safari/537.36"
}

In [47]:
def data_getting_api_rustore(w=rustore):
    pos, neg, neut = "Positive", "Negative", "Neutral"
    links_to_the_reviews_json = {
        pos: [],
        neg: []
    }

    links_to_the_reviews_csv = {
        pos: [[], [], []],
        neg: [[], [], []],
        neut: [[], [], []]
    }
    pageNum = 0
    flag = False
    while True:
        url = w.links[api] + str(pageNum)
        r = requests.get(url, headers=w.headers)
        webJson = r.json()
        for q1, q2 in webJson.items():
            if q1 == 'body':
                for item1, item2 in q2.items():
                    if item1 == 'content':
                        for rev in item2:
                            q = {'Text': '', 'Rate': 0, 'Date': ''}
                            for wh, cont in rev.items():

                                if wh == 'appRating':
                                    if cont < 3:
                                        csv_st = neg
                                    elif cont > 3:
                                        csv_st = pos
                                    else:
                                        csv_st = neut
                                    q['Rate'] = cont
                                    links_to_the_reviews_csv[csv_st][1].append(cont)

                                elif wh == 'commentDate':
                                    f = cont.split()
                                    q['Date'] = f[0]
                                    links_to_the_reviews_csv[csv_st][2].append(f[0])

                                elif wh == 'commentText':
                                    q['Text'] = cont
                                    links_to_the_reviews_csv[csv_st][0].append(cont)

                            if q['Rate'] < 3:
                                links_to_the_reviews_json[neg].append(q)
                            elif q['Rate'] > 3:
                                links_to_the_reviews_json[pos].append(q)

                    if item1 == 'totalElements':
                         if item2 == 0:
                             flag = True
                             break
                if flag:
                    break
        if flag:
            break

        pageNum += 1
    dataset_creating(links_to_the_reviews_json, links_to_the_reviews_csv, w, pos, neg)

In [48]:
def data_getting_api_sravni_ru(w=sravni_ru):
    pos, neg, neut = "Positive", "Negative", "Neutral"
    links_to_the_reviews_json = {
        pos: [],
        neg: []
    }

    links_to_the_reviews_csv = {
        pos: [[], [], []],
        neg: [[], [], []],
        neut: [[], [], []]
    }
    pageNum = 0
    while True:
        url = fr'https://www.sravni.ru/proxy-reviews/reviews/?filterBy=withRates&fingerPrint=eab9ea9d6cdaf03fcbbff091af5c0195&isClient=false&locationRoute=&newIds=true&orderBy=byDate&pageIndex={str(pageNum)}&pageSize=1000&reviewObjectId=5bb4f769245bc22a520a6353&reviewObjectType=banks&specificProductId=&withVotes=true'
        r = requests.get(url, headers=w.headers)
        webJson = r.json()
        if len(webJson['items']) == 0:
            break
        for rev in webJson['items']:
            q = {'Text': '', 'Rate': 0, 'Date': ''}
            for wh, cont in rev.items():
                if wh == 'rating':
                    if cont < 3:
                        csv_st = neg
                    elif cont > 3:
                        csv_st = pos
                    else:
                        csv_st = neut
                    q['Rate'] = cont
                    links_to_the_reviews_csv[csv_st][1].append(cont)

                elif wh == 'date':
                    f = cont.split()
                    q['Date'] = f[0]
                    links_to_the_reviews_csv[csv_st][2].append(f[0])
                elif wh == 'title':
                    q['Text'] = cont
                elif wh == 'text':
                    q['Text'] += (' ' + cont)
                    links_to_the_reviews_csv[csv_st][0].append(q['Text'])

            if q['Rate'] < 3:
                links_to_the_reviews_json[neg].append(q)
            elif q['Rate'] > 3:
                links_to_the_reviews_json[pos].append(q)

        pageNum += 1
    dataset_creating(links_to_the_reviews_json, links_to_the_reviews_csv, w, pos, neg)

In [49]:
def dataset_creating(links_to_the_reviews_json, links_to_the_reviews_csv, w, pos, neg):
    print('--- Dataset creating... ---')
    # запись в json файл
    if not os.path.exists(fr'result\api\{w.name}\json'):
        os.makedirs(fr'result\api\{w.name}\json')
    if not os.path.exists(fr'result\api\{w.name}\csv'):
        os.makedirs(fr'result\api\{w.name}\csv')

    with open(fr'result\api\{w.name}\json\revData_{w.name}.json', 'w', encoding='utf-8') as file:
        json.dump(links_to_the_reviews_json, file, indent=4, ensure_ascii=False)

    # запись в csv файл
    with open(fr'result\api\{w.name}\csv\revData_{w.name}.csv', 'w', encoding='utf-8') as file_csv, open(fr'result\api\{w.name}\csv\revData_{w.name}_pos.csv', 'w', encoding='utf-8') as file_csv_pos, open(fr'result\api\{w.name}\csv\revData_{w.name}_neg.csv', 'w', encoding='utf-8') as file_csv_neg:
        q = csv.writer(file_csv)
        q_pos = csv.writer(file_csv_pos)
        q_neg = csv.writer(file_csv_neg)

        q.writerow(
            (
                "Status",
                "Text",
                "Rating",
                "Date"
            )
        )
        q_pos.writerow(
            (
                "Text",
                "Rating",
                "Date"
            )
        )
        q_neg.writerow(
            (
                "Text",
                "Rating",
                "Date"
            )
        )
        for i in range(len(links_to_the_reviews_csv[pos][0])):
            q.writerow(
                (
                    pos,
                    links_to_the_reviews_csv[pos][0][i],
                    links_to_the_reviews_csv[pos][1][i],
                    links_to_the_reviews_csv[pos][2][i]
                )
            )
            q_pos.writerow(
                (
                    links_to_the_reviews_csv[pos][0][i],
                    links_to_the_reviews_csv[pos][1][i],
                    links_to_the_reviews_csv[pos][2][i]
                )
            )
        for i in range(len(links_to_the_reviews_csv[neg][0])):
            q.writerow(
                (
                    neg,
                    links_to_the_reviews_csv[neg][0][i],
                    links_to_the_reviews_csv[neg][1][i],
                    links_to_the_reviews_csv[neg][2][i]
                )
            )
            q_neg.writerow(
                (
                    links_to_the_reviews_csv[neg][0][i],
                    links_to_the_reviews_csv[neg][1][i],
                    links_to_the_reviews_csv[neg][2][i]
                )
            )

In [50]:
print('------ in the functions... ------')
def solve():
    data_getting_api_rustore()
    # sravni_ru.revRating = rating_getting(sravni_ru.links[websiteUrl], sravni_ru)
    #
    # q, w, e = data_getting(sravni_ru.links[websiteUrl], revTxt, True, sravni_ru)
    # sravni_ru.revTxt = [convert(q[0]), convert(q[1])]
    # sravni_ru.rating[star] = w
    # sravni_ru.links[revLinks] = [convert(e[0]), convert(e[1])]
    #
    # q, w = data_getting(sravni_ru.links[websiteUrl], revTitle, False, sravni_ru)
    # sravni_ru.revTitle = [convert(q[0]), convert(q[1])]
    #
    # q, w = data_getting(sravni_ru.links[websiteUrl], revDate, False, sravni_ru)
    # sravni_ru.revDate = [convert(q[0]), convert(q[1])]
    #
    # review_other_data(sravni_ru.revTxt, [sravni_ru.revTitle, sravni_ru.links[revLinks], sravni_ru.revDate, sravni_ru.rating[star]], sravni_ru)


    # rustore.revRating = rating_getting(rustore.links[websiteUrl], rustore)
    #
    # q, w, e = data_getting(rustore.links[websiteUrl], revContent, True, rustore)
    # rustore.revTxt = [convert(q[0]), convert(q[1])]
    # rustore.rating[star] = w
    # rustore.links[revLinks] = [convert(e[0]), convert(e[1])]
    #
    #
    # q, w = data_getting(rustore.links[websiteUrl], revDate, False, rustore)
    # rustore.revDate = [convert(q[0]), convert(q[1])]
    #review_other_data(rustore.revTxt, [[], rustore.links[revLinks], [], []], rustore)

------ in the functions... ------


In [51]:
# func activation

# for title, link in sravni_ru.revPlusLink.items():
#     rep = [',', ' ', '-', ' ']
#     for item in rep:
#         if item in title:
#             title = title.replace(item, "_")


In [52]:
# path = ''
# urles = ''
# statuses = ['positive', 'negative']
# delays = [1, 2, 1.4]
# #delays = [15, 20, 13, 18, 12.5, 13.5, 25, 12.3, 23]
#
# for website, url in rev_websites.items():
#     for status in statuses:
#         try:
#             parsing(url, status, path, [revTxt], website)
#             print(f"--- Group '{status}': success ---")
#             time.sleep(np.random.choice(delays))
#
#         except AttributeError:
#             print(f"---! Бан получен: {url}, {status} !---")
#             break
# # Блок else выполняется тогда, когда внутренний цикл НЕ прерывается
# # если внутренний цикл прерывается, то прерывается и внешний
#     else:
#         print(f"--- Website '{website.name}': success ---")
#         continue
#     break

In [36]:
solve()


# any ideas
# мини доп. функция: добавить окно, где можно будет по кластеру найти нужные отзывы: будет храниться тайтл отзыва и ссылка на него